There are two keys aspects we are looking from reviews text

- Key topics from set of reviews
- apart from just key topics contextual similarity of these reviws and hence the key topics within same context


the Attention Mechanism addresses the issue of emphasing of contextual importance of a word in different sentences.

BERT is the first popular model comes to our mind thinking of above problems hence we will use the pre trained BERT model to extract meaningful topics as required and we shall do it in following key steps

- Preprocess and clean the data using regex
- get bert topics using pre trained Model
- Extract top n keywords(2-3 grams)

In [ ]:
!pip install bertopic
!pip install flair

In [3]:
import pandas as pd
import pickle
import time
import sys
import numpy as np
from functools import partial
from copy import deepcopy

In [66]:

pd.set_option('display.max_colwidth',100)
pd.set_option('display.max_rows',100)

In [9]:
target_data=pd.read_csv('Datafiniti_Amazon_Consumer_Reviews_of_Amazon_Products.csv')
target_data.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.dateSeen,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,"2018-05-27T00:00:00Z,2017-09-18T00:00:00Z,2017-09-06T00:00:00Z,2017-09-12T00:00:00Z",False,NaN,0,3,"http://reviews.bestbuy.com/3545/5442403/reviews.htm%25252525253Fformat%25252525253Dembedded,http...",I thought it would be as big as small paper but turn out to be just like my palm. I think it is ...,Too small,llyyue,"https://www.newegg.com/Product/Product.aspx%25253FItem%25253D9SIAFVD7FK6707,https://reviews.best..."
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,"2018-05-27T00:00:00Z,2017-07-07T00:00:00Z,2017-08-09T00:00:00Z,2017-08-02T00:00:00Z,2017-08-31T0...",True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/reviews.htm%25252525253Fformat%25252525253Dembedded%2525...,This kindle is light and easy to use especially at the beach!!!,Great light reader. Easy to use at the beach,Charmi,"https://www.newegg.com/Product/Product.aspx%25253FItem%25253D9SIAFVD7FK6707,https://reviews.best..."
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,2018-05-27T00:00:00Z,True,NaN,0,4,https://reviews.bestbuy.com/3545/5442403/reviews.htm%2525252525253Fformat%2525252525253Dembedded...,"Didnt know how much i'd use a kindle so went for the lower end. im happy with it, even if its a ...",Great for the price,johnnyjojojo,"https://www.newegg.com/Product/Product.aspx%25253FItem%25253D9SIAFVD7FK6707,https://reviews.best..."
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,2018-10-09T00:00:00Z,True,177283626.0,3,5,https://redsky.target.com/groot-domain-api/v1/reviews/51467832?sort=helpfulness_desc&limit=1000&...,I am 100 happy with my purchase. I caught it on sale at a really good price. I am normally a rea...,A Great Buy,Kdperry,"https://www.newegg.com/Product/Product.aspx%25253FItem%25253D9SIAFVD7FK6707,https://reviews.best..."
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/product

In [10]:
!pip install contractions

In [11]:
import nltk
import re
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
import contractions
stop_words=stopwords.words("english")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [21]:
def clean_text(text):
    
    ## remove punctuation ###
    text=text.translate(string.punctuation)
    
    ## convert words to lower case and split them ###
    text=text.lower().split()
    
    ## remove stopwords
    stops=set(stopwords.words("english"))
    text=[w for w in text if not w in stops and len(w)>=3]
    
    text=" ".join(text)
    
    ### clean text
    
#     text=re.sub(r"[A-Za-z0-9^,!.\/+-=]"," ",text)
    text=re.sub(r"what's","what is",text)
    text=re.sub(r"\'s"," ",text)
    text=re.sub(r"\'ve","have",text)
    text=re.sub(r"i'm","i am",text)
    text=re.sub(r"n't","not",text)
    text=re.sub(r"\'re","are",text)
    text=re.sub(r"\'d","would",text)
    text=re.sub(r"\'ll","will",text)
    
    text=re.sub(r","," ",text)
    text=re.sub(r"\."," ",text)
    text=re.sub(r"!"," ! ",text)
    text=re.sub(r"\/"," ",text)
    text=re.sub(r"\^"," ",text)
    text=re.sub(r"\+"," ",text)
    text=re.sub(r"\-"," ",text)
    text=re.sub(r"\:"," ",text)
    text=re.sub(r"\;"," ",text)
    text=re.sub(r"''"," ",text)
    
    ## remove contractions
    
    text=contractions.fix(text)
    
    return text    
    
    
    
    

In [22]:
text_d=target_data['reviews.text']

In [23]:
clean_text_d=[clean_text(x) for x in text_d]

In [17]:
from bertopic import BERTopic
from flair.embeddings import TransformerDocumentEmbeddings
from sentence_transformers import SentenceTransformer, util

In [67]:
# clean_text_d

In [25]:
embedder=SentenceTransformer('bert-base-nli-stsb-mean-tokens')

Downloading:   0%|          | 0.00/391 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.98k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/630 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/409 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [26]:
bert_model=BERTopic(embedding_model=embedder,language='english',verbose=True,calculate_probabilities=True)

b_topics, bprobs=bert_model.fit_transform(documents=clean_text_d)

Batches:   0%|          | 0/157 [00:00<?, ?it/s]

2021-11-17 03:36:33,140 - BERTopic - Transformed documents to Embeddings
2021-11-17 03:37:12,256 - BERTopic - Reduced dimensionality with UMAP
2021-11-17 03:37:14,115 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [29]:
target_data['topic_id']=b_topics
target_data.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,topic_id
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,False,NaN,0,3,"http://reviews.bestbuy.com/3545/5442403/reviews.htm%25252525253Fformat%25252525253Dembedded,http...",I thought it would be as big as small paper but turn out to be just like my palm. I think it is ...,Too small,llyyue,"https://www.newegg.com/Product/Product.aspx%25253FItem%25253D9SIAFVD7FK6707,https://reviews.best...",68
1,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,True,NaN,0,5,http://reviews.bestbuy.com/3545/5442403/reviews.htm%25252525253Fformat%25252525253Dembedded%2525...,This kindle is light and easy to use especially at the beach!!!,Great light reader. Easy to use at the beach,Charmi,"https://www.newegg.com/Product/Product.aspx%25253FItem%25253D9SIAFVD7FK6707,https://reviews.best...",13
2,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,True,NaN,0,4,https://reviews.bestbuy.com/3545/5442403/reviews.htm%2525252525253Fformat%2525252525253Dembedded...,"Didnt know how much i'd use a kindle so went for the lower end. im happy with it, even if its a ...",Great for the price,johnnyjojojo,"https://www.newegg.com/Product/Product.aspx%25253FItem%25253D9SIAFVD7FK6707,https://reviews.best...",-1
3,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,True,177283626.0,3,5,https://redsky.target.com/groot-domain-api/v1/reviews/51467832?sort=helpfulness_desc&limit=1000&...,I am 100 happy with my purchase. I caught it on sale at a really good price. I am normally a rea...,A Great Buy,Kdperry,"https://www.newegg.com/Product/Product.aspx%25253FItem%25253D9SIAFVD7FK6707,https://reviews.best...",67
4,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/reviews.htm%2525252525253F

In [30]:
target_data_68=target_data[target_data['topic_id']==68]
target_data_68.head()

,id,dateAdded,dateUpdated,name,asins,brand,categories,primaryCategories,imageURLs,keys,...,reviews.doRecommend,reviews.id,reviews.numHelpful,reviews.rating,reviews.sourceURLs,reviews.text,reviews.title,reviews.username,sourceURLs,topic_id
0,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,False,NaN,0,3,"http://reviews.bestbuy.com/3545/5442403/reviews.htm%25252525253Fformat%25252525253Dembedded,http...",I thought it would be as big as small paper but turn out to be just like my palm. I think it is ...,Too small,llyyue,"https://www.newegg.com/Product/Product.aspx%25253FItem%25253D9SIAFVD7FK6707,https://reviews.best...",68
39,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,True,NaN,0,5,https://reviews.bestbuy.com/3545/5442403/reviews.htm%25252525253Fformat%25252525253Dembedded%252...,I bought this with paper white and didn‚Äôt see much of a difference in terms of resolution and ...,Excellent,Faithfy,"https://www.newegg.com/Product/Product.aspx%25253FItem%25253D9SIAFVD7FK6707,https://reviews.best...",68
77,AVqVGZNvQMlgsOJE6eUY,2017-03-03T16:56:05Z,2018-10-25T16:36:31Z,"Amazon Kindle E-Reader 6"" Wifi (8th Generation, 2016)",B00ZV9PXP2,Amazon,"Computers,Electronics Features,Tablets,Electronics,iPad & Tablets,Kindle E-readers,iPad Accessor...",Electronics,"https://pisces.bbystatic.com/image2/BestBuy_US/images/products/5442/5442403_sd.jpg,https://c1.ne...","allnewkindleereaderblack6glarefreetouchscreendisplaywifiincludesspecialoffers/b00zv9pxp2,allnewk...",...,False,152240083.0,0,4,https://redsky.target.com/groot-domain-api/v1/reviews/51467832?sort=helpfulness_desc&limit=1000&...,"It's small, light, and easy to read from. I just wish I would have spent the extra 30 bucks and ...",Should have bought paperwhite,Jessa,"https://www.newegg.com/Product/Product.aspx%25253FItem%25253D9SIAFVD7FK6707,https://reviews.best...",68
1629,AVqVGZSEQMlgsOJE6eUc,2017-03-03T16:56:06Z,2018-09-21T18:45:21Z,"Kindle E-reader - White, 6 Glare-Free Touchscreen Display, Wi-Fi - Includes Special Offers",B017JG41PC,Amazon,"Office,eBook Readers,Electronics Features,Walmart for Business,Tablets,Electronics,Amazon Ereade...","Office Supplies,Electronics","https://www.barcodable.com/images/barcode/0841667100418.png,https://i5.walmartimages.com/asr/7b8...","841667100418,0841667100418,amazonkindlewhite/5435001,amazon/53004447,kindleereaderwhite6glarefre...",...,True,NaN,0,4,https://reviews.bestbuy.com/3545/5435001/reviews.htm%253Fformat%253Dembedded%2526page%253D19,Easy to figure out and I like the paperwhite because it doesn‚Äôt strain your eyes and you can a...,"Easy on the eyes, and to use",TeaKae,"https://reviews.bestbuy.com/3545/5435001/reviews.htm%253Fformat%253Dembedded%2526page%253D40,htt...",68
1919,AVqVGZSEQMlgsOJE6eUc,2017-03-03T16:56:06Z,2018-09-21T18:45:21Z,"Kindle E-reader - White, 6 Glare-Free Touchscreen Display, Wi-Fi - Includes Special Offers",B017JG41PC,Amazon,"Office,eBook Readers,Electronics Features,Walmart for Business,Tablets,Electronics,Amazon Ereade...","Office Supplies,Electronics","https://www.barcodable.com/images/barcode/0841667100418.png,https://i5.walmartimages.com/asr/7b8...","841667100418,0841667100418,amazonkindlewhite/5435001

In [35]:
target_data=target_data.rename(columns = {'reviews.text': 'reviews'})

In [36]:
targ_df = pd.DataFrame(target_data, columns=["reviews"])
targ_df['Topic'] = b_topics
targ_df['Doc_ID'] = range(len(targ_df))
docs_per_topic = targ_df.groupby(['Topic'], as_index = False).agg({'reviews': ' '.join})

In [50]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

def c_tf_idf(documents, m, ngram_range=(2, 3)):
    count = CountVectorizer(ngram_range=ngram_range, stop_words="english").fit(documents)
    t = count.transform(documents).toarray()
    w = t.sum(axis=1)
    tf = np.divide(t.T, w)
    sum_t = t.sum(axis=0)
    idf = np.log(np.divide(m, sum_t)).reshape(-1, 1)
    tf_idf = np.multiply(tf, idf)

    return tf_idf, count
  
tf_idf, count = c_tf_idf(docs_per_topic.reviews.values, m=len(target_data))

In [51]:
def extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=20):
    words = count.get_feature_names()
    labels = list(docs_per_topic.Topic)
    tf_idf_transposed = tf_idf.T
    indices = tf_idf_transposed.argsort()[:, -n:]
    top_n_words = {label: [(words[j], tf_idf_transposed[i][j]) for j in indices[i]][::-1] for i, label in enumerate(labels)}
    return top_n_words

In [52]:
def extract_topic_sizes(df):
    topic_sizes = (df.groupby(['Topic'])
                     .reviews
                     .count()
                     .reset_index()
                     .rename({"Topic": "Topic", "reviews": "Size"}, axis='columns')
                     .sort_values("Size", ascending=False))
    return topic_sizes

In [55]:
top_n_words = extract_top_n_words_per_topic(tf_idf, count, docs_per_topic, n=5)
topic_sizes = extract_topic_sizes(targ_df); topic_sizes.head(10)

,Topic,Size
0,-1,1957
14,13,323
30,29,215
41,40,196
40,39,172
32,31,131
24,23,112
11,10,104
71,70,96
10,9,89


In [61]:
top_n_words[-1][1][0]

'easy use'

In [62]:
def tuple_to_words(tup):
    out=[]
    for i in range(len(tup)):
        
        txt=tup[i][0]
        out.append(txt)
    return out

In [63]:
tuple_to_words(top_n_words[-1])

['year old', 'easy use', 'best buy', 'battery life', 'works great']

In [65]:
review_topic_keywords=target_data[['reviews','topic_id']]
key_words=[tuple_to_words(top_n_words[x]) for x in review_topic_keywords['topic_id']]
review_topic_keywords['keywords']=key_words
review_topic_keywords.head()

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,reviews,topic_id,keywords
0,I thought it would be as big as small paper but turn out to be just like my palm. I think it is ...,68,"[page turn, turn buttons, page turn buttons, kindle voyage, turn pages]"
1,This kindle is light and easy to use especially at the beach!!!,13,"[battery life, easy use, kindle love, old kindle, love kindle]"
2,"Didnt know how much i'd use a kindle so went for the lower end. im happy with it, even if its a ...",-1,"[year old, easy use, best buy, battery life, works great]"
3,I am 100 happy with my purchase. I caught it on sale at a really good price. I am normally a rea...,67,"[buy larger, real book, download books, thousands books hated, change able reverse]"
4,Solid entry level Kindle. Great for kids. Gifted for a kid of my friend and they love to use it ...,-1,"[year old, easy use, best buy, battery life, works great]"
